In [1]:
import os
os.environ["SPARK_HOME"] = "/home/software/spark-3.1.2-bin-hadoop3.2/"
os.environ["HADOOP_CONF_DIR"] = "/usr/local/hadoop/etc/hadoop/"
os.environ["PYSPARK_PYTHON"]= "python3.9"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3.9"

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf


conf = SparkConf().setAll([('spark.app.name', 'Trabajo_taxis_1'), \
                           ('spark.executor.memory', '1g'), \
                           ('spark.executor.instances','2'), \
                           ('spark.executor.cores', '2'), \
                           ('spark.ui.port', '4064'), \
                           ('spark.driver.memory','1g')])

spark = SparkSession.builder.config(conf=conf).master('yarn').getOrCreate()

sc = spark.sparkContext


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-11-30 11:00:53,834 WARN util.Utils: Service 'SparkUI' could not bind on port 4064. Attempting port 4065.


In [2]:
#tomamos el fichero con un put lo llevamos a hdfs. luego leemos el fichero en python-spark

!/usr/local/hadoop/bin/hdfs dfs -put tripdata_2017-01.csv
df = spark.read.csv('tripdata_2017-01.csv', header=True, inferSchema=True)

put: `tripdata_2017-01.csv': File exists


PROBLEMA 1: Cantidad de datos procesados.

In [3]:
df.count() # cuenta el numero de filas del dataframe

971010

PROBLEMA 2: Esquema de datos del conjunto de datos.

In [4]:
df.printSchema() # conocer el tipo de las instancias de cada columna y si pueden ser nulas o no

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



PROBLEMA 3: Valores max/media/min de cada una de las columnas.

In [5]:
df.describe().show() # te muestra numero de filas, media, minimo y maximo de las columnas

+-------+-------------------+--------------------+---------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------------+---------------------+------------------+
|summary|           VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|   passenger_count|    trip_distance|        RatecodeID|store_and_fwd_flag|      PULocationID|     DOLocationID|      payment_type|       fare_amount|              extra|            mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|      total_amount|
+-------+-------------------+--------------------+---------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+-------------------+------------------+------------------+----------------